<center> <h1 style="background-color:darkgray; color:white" >Generating Short Stories with Mini-GPT by karpathy</h1> 
  <h2 class="list-group-item list-group-item-action active" style="background-color:darkgray; color:white" data-toggle="list"  role="tab" aria-controls="home">Kudos to Andrej-Karpathy for awesome pytorch implementation and konard for Github fork on kaggle.</h2> 

<img src = "https://i0.wp.com/syncedreview.com/wp-content/uploads/2020/05/image-116.png?resize=1024%2C565&ssl=1"/>


<a class="list-group-item list-group-item-action" data-toggle="list" href="https://github.com/karpathy/minGPT/blob/master/play_char.ipynb" role="tab" aria-controls="profile" style="color:darksgray">Notebook code has been adapted from here<span class="badge badge-primary badge-pill" style="background-color:darkslategray; color:white"></span></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!cp -r ../input/minigpt-by-andrej-karpathy/minGPT-master/mingpt/* ./

In [ ]:
from  utils import set_seed
set_seed(42)

In [ ]:
df = pd.read_csv('/kaggle/input/poe-short-stories-corpuscsv/preprocessed_data.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
all_text = list(df.text.values)
len(all_text)


In [ ]:
data = all_text
data = data[1]

In [ ]:
data

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import math
from torch.utils.data import Dataset

In [ ]:
class PoemDatset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [ ]:
block_size = 128 

In [ ]:
train_dataset = PoemDatset(data, block_size) 

In [ ]:
from model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=128)
model = GPT(mconf)

##### initialize a trainer instance and kick off training


In [ ]:
from trainer import Trainer, TrainerConfig

tconf = TrainerConfig(max_epochs=50, batch_size=128, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=128*20, final_tokens=2*len(train_dataset)*block_size)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

##### Hooray !! ,let's sample some story

In [ ]:
#character level
from utils import sample

context = "What chance, good lady"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)


 <h2 class="list-group-item list-group-item-action active" style="background-color:darkgray; color:white" data-toggle="list"  role="tab" aria-controls="home">Cheers!!!!</h2>

<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:darkgray; color:white" data-toggle="list"  role="tab" aria-controls="home">References</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="https://github.com/karpathy/minGPT" role="tab" aria-controls="profile" style="color:darksgray">Mini-GPT<span class="badge badge-primary badge-pill" style="background-color:darkslategray; color:white"></span></a>